Known Elements and Energies:

A dictionary, elements, is given where known elements (like Cu, Fe, Zn) are associated with their respective energies. This aids in identifying and labeling peaks in the spectrum.
Reading Data:

The read_data function reads a CSV file and extracts the energy and counts columns.
Peak Detection:

There are two methods implemented for peak detection:
Threshold Method: This is a simple method where a peak is identified if its count is greater than a user-defined threshold and is also greater than its neighboring values. This method may not be highly accurate for complex data.
Scipy Method: This utilizes the find_peaks function from the scipy.signal module, which is more sophisticated and can accurately detect peaks using various properties, including height.
Quantification:

For each detected peak, the quantify_peaks function computes the area by summing the counts in a range around the peak. The range is hardcoded as 5 units before and after the peak. This quantified data helps in understanding the prominence or intensity of each peak.
Annotating Peaks:

The annotate_peaks function labels the detected peaks with the closest known element from the elements dictionary.
Data Adjustments:

Users can introduce noise, adjust peak heights, and smooth the data:
Noise: Random noise is added based on a normal distribution.
Peak Height: The counts are multiplied by a user-defined factor to simulate a change in peak height.
Smoothing: The Savitzky-Golay filter, a polynomial smoothing method, is applied to the data. This can help in reducing noise and making peaks more distinguishable.
Background Subtraction:

To differentiate true peaks from the background noise or continuous contributions, a background subtraction method is implemented.
The function subtract_background fits a polynomial (cubic by default) to the regions of the spectrum assumed to be just background. Once the background is estimated, it's subtracted from the original data to emphasize the peaks.
Interactive Widgets:

The code provides an interactive GUI using ipywidgets. Users can upload data, select peak detection methods, set thresholds, introduce noise, adjust peak heights, smooth data, and visualize results in real-time.
Visualization:

Matplotlib is used for plotting. The plots help visualize the original data, detected peaks, background subtraction, and quantified peaks.

In [1]:
"""
XRF Analysis and Visualization

This module provides utilities and a simple GUI for analyzing X-ray fluorescence (XRF) data.
The functionality provided includes reading data, detecting peaks, quantifying peaks, and visualizing results.

Dependencies:
    - numpy
    - pandas
    - matplotlib
    - scipy
    - ipywidgets
    - IPython

Attributes:
    elements (dict): A dictionary containing known elements and their respective energies.

Functions:
    read_data(file_name): Reads a CSV file and returns the energy and counts.
    
    threshold_detect_peaks(data, threshold): Detects peaks in the data using a simple threshold method.
    
    scipy_detect_peaks(data, threshold): Detects peaks using the Scipy library.
    
    quantify_peaks(energy, counts, peaks): Quantifies the area under detected peaks.
    
    annotate_peaks(ax, energies, counts): Annotates detected peaks with closest known element symbols.
    
    adjust_data(energy, counts): Adjusts the data based on GUI parameters such as noise, peak height, and smoothing.
    
    on_file_upload(change): Handles file upload and displays the uploaded data with optional modifications.
    
    on_plot_and_quantify(button): Visualizes and quantifies the data based on user-selected parameters.
    
    polynomial_fit(x, a, b, c, d): A third-degree polynomial fit function.
    
    subtract_background(energy, counts, degree=3): Subtracts background from the spectrum.
    
    on_background_subtraction(button): Visualizes data with background subtraction.

Widgets:
    uploader (FileUpload): Allows users to upload a CSV file containing XRF data.
    
    method_dropdown (Dropdown): Allows users to select the peak detection method.
    
    threshold_slider (FloatSlider): Enables setting a threshold for peak detection.
    
    noise_slider (FloatSlider): Adjusts the noise level in the data.
    
    peak_height_slider (FloatSlider): Modifies the peak heights in the data.
    
    smoothing_slider (IntSlider): Determines the window size for data smoothing.
    
    plot_button (Button): Initiates data plotting and quantification.
    
    bg_subtract_button (Button): Subtracts the background from the spectrum.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter
import ipywidgets as widgets
from IPython.display import display, clear_output

# Known elements and their energies
elements = {
    'Cu': 8.0,
    'Fe': 6.4,
    'Zn': 9.6
    # Add more elements as necessary
}

def read_data(file_name):
    data = pd.read_csv(file_name)
    return data['energy'].values, data['counts'].values

def threshold_detect_peaks(data, threshold):
    peaks = []
    for i in range(1, len(data)-1):
        if data[i] > threshold and data[i-1] < data[i] and data[i+1] < data[i]:
            peaks.append(i)
    return peaks

def scipy_detect_peaks(data, threshold):
    peaks, _ = find_peaks(data, height=threshold)
    return peaks

def quantify_peaks(energy, counts, peaks):
    quantified = {}
    for peak in peaks:
        area = np.sum(counts[peak-5:peak+5])
        quantified[energy[peak]] = area
    return quantified

def annotate_peaks(ax, energies, counts):
    for energy, count in zip(energies, counts):
        closest_element = min(elements.keys(), key=lambda k: abs(elements[k]-energy))
        ax.annotate(closest_element, (energy, count), fontsize=10, xytext=(5,-5), textcoords='offset points')

uploader = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload XRF Data'
)

output = widgets.Output()

method_dropdown = widgets.Dropdown(
    options=['Threshold', 'Scipy'],
    value='Threshold',
    description='Method:',
    disabled=False
)

threshold_slider = widgets.FloatSlider(
    value=0.1,        # Default value
    min=0.001,       # Minimum value
    max=10.0,        # Maximum value
    step=0.001,      # Adjust this based on the granularity you need
    description='Threshold:',
    continuous_update=False
)


noise_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=3.0,
    step=0.1,
    description='Noise Level:',
    continuous_update=False
)

peak_height_slider = widgets.FloatSlider(
    value=1.0,   # Default multiplier (no change)
    min=0.5,    # Minimum multiplier 
    max=5.0,    # Maximum multiplier
    step=0.1,  
    description='Peak Multiplier:',
    continuous_update=False
)


smoothing_slider = widgets.IntSlider(
    value=5,      # Default window size
    min=1,        # Minimum window size
    max=51,       # Maximum window size
    step=2,       # It's stepped by 2 to ensure window size is always odd.
    description='Smoothing Window:',
    continuous_update=False,
    tooltip="Higher value means more smoothing. Must be an odd integer."
)


plot_button = widgets.Button(
    description='Plot and Quantify',
    disabled=True,
    button_style='success',  # This makes the button dark green.
    tooltip='Plot and Quantify',
    icon='check'
)

def adjust_data(energy, counts):
    # Adjust noise
    counts += np.random.normal(0, noise_slider.value, energy.shape)
    
    # Adjust peak height
    counts *= peak_height_slider.value
    
    # Apply smoothing
    if smoothing_slider.value > 1:
        counts = savgol_filter(counts, smoothing_slider.value, 3)
    
    return energy, counts

def on_file_upload(change):
    with output:
        clear_output(wait=True)
        uploaded_file = uploader.value[0]
        content = uploaded_file['content']
        with open('temp.csv', 'wb') as f:
            f.write(content)
        energy, counts = read_data('temp.csv')
        energy, counts = adjust_data(energy, counts)  # Adjust the data
        plt.plot(energy, counts, label='Modified Data')
        plt.legend()
        plt.show()
    plot_button.disabled = False
    bg_subtract_button.disabled = False  # Enable the bg_subtract_button when a file is uploaded


def on_plot_and_quantify(button):
    with output:
        clear_output(wait=True)
        energy, counts = read_data('temp.csv')
        energy, counts = adjust_data(energy, counts)  # Adjust the data
        
        if data_source_dropdown.value == 'Background Subtracted Data':
            energy, counts = subtract_background(energy, counts)

        if method_dropdown.value == 'Threshold':
            peaks = threshold_detect_peaks(counts, threshold_slider.value)
        else:
            peaks = scipy_detect_peaks(counts, threshold_slider.value)

        quantified = quantify_peaks(energy, counts, peaks)
        
        fig, axes = plt.subplots(1, 2, figsize=(12,6))
        ax1, ax2 = axes
        ax1.plot(energy, counts, label='Data')
        ax1.scatter(energy[peaks], counts[peaks], color='red', label='Detected Peaks')
        ax1.legend()
        ax1.set_title('Data with Peaks')
        ax2.plot(list(quantified.keys()), list(quantified.values()), 'o-', label='Quantified Peaks')
        annotate_peaks(ax2, list(quantified.keys()), list(quantified.values()))
        ax2.legend()
        ax2.set_title('Quantified Data')
        plt.tight_layout()
        plt.show()


from scipy.optimize import curve_fit

def polynomial_fit(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

def subtract_background(energy, counts, degree=3):
    non_peak_regions = [(0, 50), (150, 200), (350, 400)]
    x_background = []
    y_background = []
    
    for start, end in non_peak_regions:
        x_background.extend(energy[start:end])
        y_background.extend(counts[start:end])

    if degree == 3:
        popt, _ = curve_fit(polynomial_fit, x_background, y_background)
        background = polynomial_fit(energy, *popt)
    else:
        z = np.polyfit(x_background, y_background, degree)
        p = np.poly1d(z)
        background = p(energy)

    counts_corrected = counts - background
    return energy, counts_corrected


uploader.observe(on_file_upload, names='value')
plot_button.on_click(on_plot_and_quantify)

# Add a new button for Background Subtraction
bg_subtract_button = widgets.Button(
    description='Subtract Background',
    disabled=True,
    button_style='', 
    tooltip='Subtract the Background from the Spectrum'
)

data_source_dropdown = widgets.Dropdown(
    options=['Original Data', 'Background Subtracted Data'],
    value='Original Data',
    description='Data Source:',
    disabled=False
)

def on_background_subtraction(button):
    with output:
        clear_output(wait=True)
        energy, counts = read_data('temp.csv')
        energy, counts_corrected = subtract_background(energy, counts)
        
        plt.figure(figsize=(8,6))
        plt.plot(energy, counts, label='Original Data')
        plt.plot(energy, counts_corrected, label='Background Subtracted', color='red')
        plt.legend()
        plt.title('Background Subtraction')
        plt.show()

bg_subtract_button.on_click(on_background_subtraction)


display_widgets = widgets.VBox([
    uploader,
    method_dropdown,
    widgets.HBox([threshold_slider, noise_slider, peak_height_slider]),
    smoothing_slider,
    plot_button,
    output
])

top_row = widgets.HBox([uploader, data_source_dropdown, plot_button])  # This places the widgets side by side.

display_widgets = widgets.VBox([
    top_row,
    method_dropdown,
    widgets.HBox([threshold_slider, noise_slider, peak_height_slider]),
    smoothing_slider,
    bg_subtract_button,
    output
])


display(display_widgets)

